# **Przygotowanie środowiska**

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Przygotowanie danych**

In [ ]:
X = np.load('/content/drive/MyDrive/finalne_zdjecia.npy')
y_age = np.load('/content/drive/MyDrive/finalny_wiek.npy')

In [ ]:
torch_zdjecia = torch.from_numpy(X)
torch_etykiety = torch.from_numpy(y_age)

In [ ]:
# Wykorzystanie Cuda
device = torch.device("cuda")
zdjecia = torch_zdjecia.to(device)
etykiety = torch_etykiety.to(device)


# Definiowanie niestandardowego zbioru danych
class ZbiorDanych(Dataset):
    def __init__(self, zdjecia, etykiety):
        self.zdjecia = zdjecia
        self.etykiety = etykiety

    def __len__(self):
        return len(self.zdjecia)

    def __getitem__(self, idx):
        obraz = self.zdjecia[idx].to(device)
        etykieta = self.etykiety[idx].to(device)
        return obraz, etykieta

#  Przekształcenie etykiet wiekowych
def wiek_na_klase(wiek):
    if wiek <= 5:
        return 0
    elif wiek <= 10:
        return 1
    elif wiek <= 15:
        return 2
    elif wiek <= 20:
        return 3
    elif wiek <= 30:
        return 4
    elif wiek <= 40:
        return 5
    elif wiek <= 50:
        return 6
    elif wiek <= 70:
        return 7
    else:
        return 8

# Instancja ZbiorDanych
przeksztalcone_etykiety = np.array([wiek_na_klase(wiek) for wiek in etykiety])
przeksztalcone_etykiety = torch.tensor(przeksztalcone_etykiety, dtype=torch.long).to(device)
dataset = ZbiorDanych(zdjecia, przeksztalcone_etykiety)

# Podział na zbiory treningowe i testowe
rozmiar_testowy = int(0.2 * len(dataset))
rozmiar_treningowy = len(dataset) - rozmiar_testowy
dataset_treningowy, dataset_testowy = random_split(dataset, [rozmiar_treningowy, rozmiar_testowy])

# **Trenowanie modelu**

In [ ]:
# Ładowanie danych
loader_treningowy = DataLoader(dataset_treningowy, batch_size=32, shuffle=True)
loader_testowy = DataLoader(dataset_testowy, batch_size=32, shuffle=False)

# Ustawienie modelu
model = models.resnet34(pretrained=True).to(device)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 9) # liczba_klas
model = model.to(device)

# Kryteria, optymalizator
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trenowanie modelu
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()  # Ustawienie modelu na tryb treningowy
        running_loss = 0.0

        for inputs, labels in loader_treningowy:
            inputs, labels = inputs.to(device), labels.to(device)  # Przesyłanie danych na GPU
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {running_loss/len(loader_treningowy)}')


    return model

model = train_model(model, criterion, optimizer, num_epochs=5).to(device)

# **Ocena dokładności**

In [ ]:
# Ocena modelu
def evaluate_model(model, loader_testowy):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in loader_testowy:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')
# Ocena modelu na zbiorze testowym
evaluate_model(model, loader_testowy)

# **Zastosowanie modelu**

In [ ]:
zdjecie1 = '/content/drive/My Drive/14.jpg'
zdjecie2 = '/content/drive/My Drive/26.jpg'
zdjecie3 = '/content/drive/My Drive/48.jpg'


def modify_image(image_path):
  image = Image.open(image_path)
  image = image.convert('L')
  single_image = np.array(image)
  single_image = single_image / 255.0
  padding = 12  # (224 - 200) / 2
  padded_image = np.pad(single_image, ((padding, padding), (padding, padding)), 'constant', constant_values=1)
  image_3_channels = np.repeat(padded_image[:, :, np.newaxis], 3, axis=2)
  final_array = np.transpose(image_3_channels, (2, 0, 1))
  return torch.tensor(final_array).float()

def predict(zdjecie):
  model.eval()
  with torch.no_grad():
      zdjecie = zdjecie.unsqueeze(0)
      zdjecie = zdjecie.to(device)
      outputs = model(zdjecie)
      _, predykcja = torch.max(outputs, 1)
  return predykcja

test = modify_image(zdjecie3).to(device)
test1 = predict(test)
print(test1)